# Homework 2

### DUE: 02/16/2017 before class at 10:30am

The objective of this homework is for gaining experiences in designing *streaming* algorithms. Similar to Homework 1, we’re going to use the Citibike data sets posted on NYU Classes. In addition to **citibike.csv** we will also use **citibike_docking_events.csv**. Both of the data sets can be downloaded from the NYU Classes's Resources page. You are required to turn in this notebook with all the parts filled in place of <###>. Your notebook must be named BDM\_HW2\_**NetId**.ipynb.



## Task 1 (6 points)

Your task is to **compute the maximum number of active "citibikers"** that were using the Citibike service at any point in time. This the same as computing the maximum number of citibikes that were checked out at a particular time. The input data set is **citibike_docking_events.csv**, which logged all docking and undocking events at all Citibike stations. The description of the fields in this file is as follows:

|Column name|Description|
|--|--|
|time |The timestamp of the event. All events are sorted increasingly by their timestamps. |
|bikeid |The unique ID of the bike involved in this event. |
|station_id |The station ID, where the event happened. |
|event |A string of either *"dock"* or *"undock"* for describing the drop-off or pick-up event, respectively. |

For example, let's assume that on *Feb-01-2015*, there was a user that picked a bike at the station ID *521* at midnight and dropped it at the station ID *423* at 14 minutes past midnight. If the bike that this customer used has the ID of *17131*, then you should see two events being logged in this data set as:

<pre>
...
2015-02-01 00:00:00+00,17131,521,undock
...
2015-02-01 00:14:00+00,17131,423,dock
...
</pre>

You are given the above data set in a streaming fashion (reading in row by row), and must design a streaming algorithm that uses the least possible additional memory to compute the maximum number of active users of the Citibike service. Please modify the code snippet below to complete this task. Your code should only output a single number, which is the number of active users. Of course, you can add additional initialization codes outside of the for loop as needed.
Additional, please provide a brief rationale and/or justification for your design after the code.

In [4]:
import csv

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row

maxActiveUsers = 0
current_count = 0
for row in csvRows('citibike_docking_events.csv'):
    
    # increment or decrement the current count
    if row['event'] == 'dock':
        current_count -= 1
    else:
        current_count += 1
    
    # if the current count exceeds the known max, increment the max
    if current_count > maxActiveUsers:
        maxActiveUsers +=1
    
print 'max active:', maxActiveUsers

max active: 250


#### RATIONALE AND JUSTIFICATION

Because we don't know what happens BEFORE or AFTER the window for this dataset, all we can calculate is the relative difference in ridership at any given moment, with respect to to the starting point of the data set. Alternatively, one could maintain a list of the bike id's in use, and if there is a 'docking' record without a corresponding 'undocking' event preceeding it, then the docking would be ignored. This would effectively set t=0 at the start of the data file. This would be memory hungry, and is unnecessary for the above problem.

The above approach is very simple - any time a bike docks, the count decrements, and any time a bike undocks, the count increments. Whenever the real-time count exceeds the existing max_count, a new max value is set. Because of the time-sorted nature of the data, this simple approach works.

## Task 2 (4 points)

The objective of this task is identical to Task 1's but you are asked to use the **cibibike.csv** data set instead of the docking events. The main difference (and challenge) is that both pick-up and drop-off event for each trip is now presented as a single record, thus, the drop-off events are not sorted by their timestamps. You are again asked to do this in a streaming fashion that needs to minimize the amount of memory usage. Please modify the code below accordingly, and also with a brief explaination of the solution.

In [28]:
import csv
import dateutil


def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row

            
maxActiveUsers = 0
stop_buffer = []
for row in csvRows('citibike.csv'):
    
    # parse date strings
    start = dateutil.parser.parse(row['starttime'][:-3])
    stop = dateutil.parser.parse(row['stoptime'][:-3])
    
    # add new record to a sorted buffer of relevant stop times
    stop_buffer.append(stop)
    stop_buffer.sort()
    
    # how many ongoing trips become irrelevant with the addition of the newest record?
    # note that we're using '<=' instead of '<' under the assumption that if a trip stops at the
    # same second that another trip starts, those two trips did NOT coincide
    delta = len(filter(lambda x: x <= start, stop_buffer))
    
    # update the buffer by removing the 'delta' lowest stoptimes
    stop_buffer = stop_buffer[delta:]
    
    # increment user count if we can
    if len(stop_buffer) > maxActiveUsers:
        maxActiveUsers += 1
        
print 'max active: ', maxActiveUsers

max active:  250


#### RATIONALE AND JUSTIFICATION

The above method is the "absolute" one - it provides an exact solution, but the buffer list has to grow as big as the max_active_users number gets. Alternatively (to potentially save memory), we could limit the size of the buffer (it only needs to be large enough to be to handle the highest values of 'delta'), and maintain both a realtime count and max_user count as integers. This would be a more complex algorithm, but likely less memory-hungry. 

As it is, this approach maintains a list of stop times that are "currently relevant" - meaning that they each correspond to a bike trip that is still in progress as the current data record is read. Whenever a new trip comes in with a start time that is sufficiently late to make some of the memory-stored trips irrelevant, those irrelevant trips are bumped from the buffer.

This process is repeated for the whole stream of data, and the max_active_users number is simply the size of the buffer when it reached its peak.